In [ ]:
from oxford_pets_train_script import *
import torchvision.transforms.functional as F
import torch
import numpy as np
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
from torchvision.io import read_image
from pathlib import Path
from torchvision.utils import draw_bounding_boxes, draw_segmentation_masks
from torchvision.transforms.functional import convert_image_dtype

# https://pytorch.org/vision/0.10/auto_examples/plot_visualization_utils.html
# boxes and masks

In [ ]:
def show(imgs):
    if not isinstance(imgs, list):
        imgs = [imgs]
    fix, axs = plt.subplots(ncols=len(imgs), squeeze=False)
    for i, img in enumerate(imgs):
        img = img.detach()
        img = F.to_pil_image(img)
        axs[0, i].imshow(np.asarray(img))
        axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])

mean=-np.array([0.485, 0.456, 0.406])
std= 1/np.array([0.229, 0.224, 0.225])
inverse_norm= transforms.Compose([transforms.Normalize(mean=[0.0,0.0,0.0], std=std), transforms.Normalize(mean=mean, std=[1,1,1])])

def draw_bb(normed_img_data, boxes):
    reg_img = inverse_norm(normed_img_data)
    reg_img = convert_image_dtype(reg_image,dtype=torch.uint8).to("cpu")
    boxed_img = draw_bounding_boxes(reg_image, boxes)
    return boxed_img
    


In [ ]:
fs, ds, ts = gen_dataset()

fs, ds, ts = gen_dataset()

model = torch.load("checkpoint_13").to(device)
tdata, tlabel = ts[100]

model.eval()
with torch.no_grad():
    preds = model([tdata])

reg_image = convert_image_dtype(inverse_norm(tdata.clone()),dtype=torch.uint8).to("cpu")
reg_image_w_boxes = draw_bounding_boxes(reg_image, preds[0]["boxes"][:3])
reg_image_w_mask = draw_segmentation_masks(reg_image, masks=preds[0]["masks"][0]>0.5, alpha=0.7)
reg_image_w_box_w_mask = draw_bounding_boxes(draw_segmentation_masks(reg_image, masks=preds[0]["masks"][0]>0.5, alpha=0.7), preds[0]["boxes"][:1])


In [ ]:
show([reg_image_w_mask, draw_bb(tdata, preds[0]["boxes"][:3])])